##### Por hacer:

1.- Encontrar mejor combinación

2.- Encontrar número máximo de generaciones

-Utilidad por cerrar ciclos tiene muy poco peso en los últimos semestres

##### Notas

Se debería añadir utilidad al llevar materias básicas sobre las de elección libre?

-Al darle un peso mayor a la utilidad de cerrar ciclos, ¿mejorarán los resultados?

-¿Por qué el mejor resultado teórico es mejor que el resultado práctico a pesar de tener mejor desempeno?

El desempeño de la poblacion siempre se estanca porque terminan habiendo muchas cargas inválidas

La utilidad de cerrar ciclos no tiene suficiente relevancia sobre las otras funciones debido a la naturaleza de su cálculo.

##### Cambios:

-Se modificó "obtenerOfertaUtil" para excluir materias de elección libre ya aprobadas de primeros 4 semestres.

-Se cambio función de utilidad disponibilidad de horario

-Error en "RespetaSeriacion", si la materia necesaria ya había sido aprobada retornaba false

In [ ]:
import pandas as pd
import numpy as np
import pygad as pg
import time

# Limpieza y preparación de los datos

## Lectura de datos

In [ ]:
d16 = pd.read_csv('./datos/datos_2016.csv')
d17 = pd.read_csv('./datos/datos_2017.csv')
d18 = pd.read_csv('./datos/datos_2018.csv')
d19 = pd.read_csv('./datos/datos_2019.csv')
d2021 = pd.read_csv('./datos/2020_2021.csv')

In [ ]:
asignaturas = pd.read_csv('asignaturas2.csv',encoding = 'ISO-8859-1')

In [ ]:
plan_2016 = asignaturas.query('plan == "IDP2016"')

In [ ]:
oferta = pd.read_csv('oferta_ideio.csv',encoding = 'ISO-8859-1')

In [ ]:
seriaciones = pd.read_csv('seriacion.csv')

## Visualización de datos

In [ ]:
d16.head()

In [ ]:
d2021.head()

In [ ]:
asignaturas.head()

In [ ]:
plan_2016.head()

In [ ]:
oferta.head()

In [ ]:
seriaciones.head()

## Limpieza de datos

In [ ]:
cargas16_19 = pd.concat([d16,d17,d18,d19])
cargas20_21 = d2021

In [ ]:
cargas16_19 = cargas16_19.query('carrera == "Ingeniería en Datos e Inteligencia Organizacional"')

In [ ]:
cargas16_19 = cargas16_19[['Unnamed: 0', 'carrera', 'periodo', 'clave', 'asignatura',
       'promediofinal', 'docente', 'ceveval_global', 'ceneval_analitico',
       'ceneval_matematico', 'ceneval_lengua', 'ceneval_esp',
       'n_matricula']]

In [ ]:
matriculas16 = cargas16_19.query('n_matricula > 160000000 and n_matricula < 170000000')['n_matricula'].unique()
matriculas17 = cargas16_19.query('n_matricula > 170000000 and n_matricula < 180000000')['n_matricula'].unique()
matriculas18 = cargas16_19.query('n_matricula > 180000000 and n_matricula < 190000000')['n_matricula'].unique()
matriculas19 = cargas16_19.query('n_matricula > 190000000 and n_matricula < 200000000')['n_matricula'].unique()
matriculas20 = cargas20_21.query('n_matricula > 200000000 and n_matricula < 210000000')['n_matricula'].unique()
matriculas21 = cargas20_21.query('n_matricula > 210000000 and n_matricula < 220000000')['n_matricula'].unique()

In [ ]:
matriculas = np.concatenate((matriculas16, matriculas17,matriculas18,matriculas19,matriculas20,matriculas21))

In [ ]:
cargas20_21 = cargas20_21.rename(columns={'programa': 'carrera'})

In [ ]:
formato_cargas20_21 = cargas20_21[['carrera', 'periodo', 'clave', 'asignatura','docente','n_matricula','promediofinal']]
formato_cargas16_19 = cargas16_19[['carrera', 'periodo', 'clave', 'asignatura','docente','n_matricula','promediofinal']]

formato_cargas = pd.concat([formato_cargas16_19,formato_cargas20_21])

In [ ]:
oferta = oferta[['Clave','Nombre', 'Maestro', 'Lunes','Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado']]

In [ ]:
oferta['Lunes'].unique()

In [ ]:
oferta['Martes'].unique()

In [ ]:
oferta['Miercoles'].unique()

In [ ]:
oferta['Jueves'].unique()

In [ ]:
oferta['Viernes'].unique()

In [ ]:
oferta.loc[36,'Miercoles'] = '13:00-15:00'

In [ ]:
oferta['Miercoles'].unique()

In [ ]:
def formatoHora(hora):
    if len(hora) == 11:
        return hora
    elif len(hora) == 10:
        return '0' + hora
    elif len(hora) == 9:
        return '0' + hora[0:5] + '0' + hora[5:]
    elif len(hora) == 1:
        return '-'

In [ ]:
oferta['Lunes'] = oferta['Lunes'].apply(lambda horas: formatoHora(horas))
oferta['Martes'] = oferta['Martes'].apply(lambda horas: formatoHora(horas))
oferta['Miercoles'] = oferta['Miercoles'].apply(lambda horas: formatoHora(horas))
oferta['Jueves'] = oferta['Jueves'].apply(lambda horas: formatoHora(horas))
oferta['Viernes'] = oferta['Viernes'].apply(lambda horas: formatoHora(horas))

# Definición de funciones útiles

## Función para obtener un Kardex aleatorio

In [ ]:
def obtenerKardex(mat = -1):
    if(mat == -1):
        mat = np.random.choice(matriculas)
    return formato_cargas.query('n_matricula == ' + str(mat)).sort_values('periodo')

In [ ]:
def obtenerCreditos(kardex):
    claves = kardex.query('promediofinal >= 7')['clave'].unique()
    totalCreditos = 0
    for clave in claves:
        if(clave[0:2] == 'LI' or clave[0:2] == 'TA' or clave[0:2] == 'AD'):
            continue
        totalCreditos += plan_2016.query('clave == "' + clave + '"')['creditos'].values[0]
    return totalCreditos

## Obtención de oferta útil

En este sección se crea la función "obtenerOfertaUtil" la cual filtra la oferta académica para eliminar las materias que no cumplen con la restricción de llevar materias aprobadas y la restricción de llevar materias que no respeten la seriación.

In [ ]:
def materiaHaSidoAprobada(kardex,clave):
    if len(kardex.query('clave == "' + clave + '" and promediofinal >= 7')) == 0:
        return False
    else:
        return True

In [ ]:
def respetaSeriacion(clave,kardex):
    if len(seriaciones.query('ser2 == "' + clave + '"')) == 0:
        return True
    else:
        #Se guarda en necesarias la lista de asignaturas necesarias para llevar la asignura "clave"
        necesarias = seriaciones.query('ser2 == "' + clave + '"')['ser1'].unique()
        
        #Se recorre la lista para comprobar si el estudiante ya pasó las asignaturas necesarias, en caso de que le falte 
        #al menos una, entonces no se respeta la seriación.
        for necesaria in necesarias:
            if not(materiaHaSidoAprobada(kardex,necesaria)):
                return False
        return True

In [ ]:
#Filtra las materias que no cumplen las restricciones 1 y 2

def obtenerOfertaUtil(kardex,oferta):
    #Se eliminan las materias en la oferta que ya han sido aprobadas
    aprobadas = kardex.query('promediofinal >= 7')['clave'].unique()
    for clave in aprobadas:
        oferta = oferta.query('Clave != "' + clave + '"')
        
    #Se eliminan las materias que el alumno no puede llevar por la seriación
    ofertaUtilIndex = set(oferta.index.values)
    for idx in oferta.index:
        if not(respetaSeriacion(oferta.loc[idx]['Clave'],kardex)):
            ofertaUtilIndex.remove(idx)
            
    #Se eliminan prácticas profesionales y proyecto terminal
    for i in range(len(oferta['Clave'].values)):
        clave = oferta['Clave'].values[i]
        if clave[0:3] == 'PID' or clave == 'IT0427':
            try:
                ofertaUtilIndex.remove(oferta.index.values[i])
            except:
                continue
    ofertaUtil = oferta.loc[list(ofertaUtilIndex)]
    
    #Se eliminan materias de elección libre de primer y segundo ciclo
    if(materiaHaSidoAprobada(kardex,'IL0102')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0160"')
    if(materiaHaSidoAprobada(kardex,'ID0160')):
        ofertaUtil = ofertaUtil.query('Clave != "IL0102"')
    
    if(materiaHaSidoAprobada(kardex,'IT0103')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0161"')
    if(materiaHaSidoAprobada(kardex,'ID0161')):
        ofertaUtil = ofertaUtil.query('Clave != "IT0103"')
        
    if(materiaHaSidoAprobada(kardex,'ID0264')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0262"')
    if(materiaHaSidoAprobada(kardex,'ID0262')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0264"')
        
    if(materiaHaSidoAprobada(kardex,'ID0263')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0265"')
    if(materiaHaSidoAprobada(kardex,'ID0265')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0263"')
        
    
    return ofertaUtil

In [ ]:
oferta.query('Clave != "ID0161"')

## Generación de carga aleatoria válida

#### NOTA: Tomar en cuenta materias repetidas y por ahora descartar talleres deportivos, artísticos, lengua inglesa y prácticas profesionales

En esta sección creamos las funciones "generarCargaValida" la cual retorna una string de bits representando una carga académica con base a la oferta útil. También se crea la función "obtenerDatosCarga" que nos ayuda a obtener los datos de una carga académica con base a la oferta útil y a una string de bits que denominamos "bits de carga"

In [ ]:
def generarSolucionAleatoria(ofertaUtil):
    solucion = [-1] * 9
    for i in range(9):
        gen = np.random.randint(-1,len(ofertaUtil))
        if gen == -1:
            solucion[i] = -1
        else:
            solucion[i] = ofertaUtil.index[gen]
    return solucion

In [ ]:
def obtenerDatosCarga(solucion,ofertaUtil):
    solucionU = np.array(solucion)
    solucionU = np.unique(solucionU)
    solucionU = solucionU[solucionU >= 0]
    
    return ofertaUtil.loc[solucionU]

In [ ]:
def comprobarTraslapacion(solucion,ofertaUtil):
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado']
    
    for dia in dias:
        horarioDia = datosCarga.sort_values(dia)[dia].values
        for i in range(len(horarioDia)):
            if horarioDia[i] == '-':
                continue
            horaInicioI = int(horarioDia[i][0:2])
            horaFinI = int(horarioDia[i][6:8])
            
            for j in range(i+1,len(horarioDia)):
                horaInicioJ = int(horarioDia[j][0:2])
                horaFinJ = int(horarioDia[j][6:8])
                
                if not(horaFinJ  <= horaInicioI or horaInicioJ >= horaFinI):
                    return True
    return False

In [ ]:
def esValido(solucion,ofertaUtil):
    #Si se repite una materia es inválido
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    if len(datosCarga['Clave'].unique()) < len(datosCarga):
        return False
    #Si se traslapan dos materias es inválido
    if comprobarTraslapacion(solucion,ofertaUtil):
        return False
    return True

In [ ]:
def generarSolucionValida(ofertaUtil):
    bitsDeCarga = generarSolucionAleatoria(ofertaUtil)
    
    while not(esValido(bitsDeCarga,ofertaUtil)):
        bitsDeCarga = generarSolucionAleatoria(ofertaUtil)
    
    return bitsDeCarga

In [ ]:
def obtenerHorario(carga,ofertaUtil):
    primeraHoraMinima = 24
    ultimaHoraMaxima = 0
    datosCarga = obtenerDatosCarga(carga,ofertaUtil)
    horario = pd.DataFrame({
        'Hora': ['7:00-8:00','8:00-9:00','9:00-10:00','10:00-11:00','11:00-12:00','12:00-13:00','13:00-14:00','14:00-15:00','15:00-16:00','16:00-17:00','17:00-18:00','18:00-19:00','19:00-20:00','20:00-21:00','21:00-22:00'],
        'Lunes': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Martes': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Miercoles': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Jueves': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Viernes': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Sabado': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
    })
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado']
    for dia in dias:
        for i in range(len(datosCarga)):
            if datosCarga[dia].iloc[i] == '-':
                continue

            horaInicio = int(datosCarga.iloc[i][dia][0:2])
            horaFin = int(datosCarga.iloc[i][dia][6:8])
            
            primeraHoraMinima = min(primeraHoraMinima,horaInicio)
            ultimaHoraMaxima = max(ultimaHoraMaxima,horaFin)
            
            nombre = datosCarga.iloc[i]['Nombre']

            for hora in range(horaInicio,horaFin):
                horario.loc[hora-7,dia]=nombre
    return horario[(primeraHoraMinima-7):(ultimaHoraMaxima-6)]

# Definición de funciones de utilidad y de costo

## Utilidad de carga académica con base en las materias reprobadas

In [ ]:
def obtenerMateriasReprobadas(kardex):
    materiasReprobadas = kardex.query('promediofinal < 7')['clave'].unique()
    materiasReprobadasFinal = []
    for clave in materiasReprobadas:
        aprobado = len(kardex.query('clave == "' + clave + '" and promediofinal >= 7'))
        if aprobado == 0:
            materiasReprobadasFinal.append(clave)
    return materiasReprobadasFinal

In [ ]:
def UpMR(kardex,solucion,ofertaUtil):
    materiasReprobadas = obtenerMateriasReprobadas(kardex)
    if len(materiasReprobadas) == 0:
        return 0
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    utilidadTotal = 0
    
    for clave in materiasReprobadas:
        cargado = len(datosCarga.query('Clave == "' + clave + '"'))
        if cargado == 1:
            utilidadTotal += 1
            
    
    #Normalización
    utilidad = (utilidadTotal)/(len(materiasReprobadas))
    return utilidad

## Utilidad de carga académica con base en el cierre de ciclos

In [ ]:
def UpCC(solucion,ofertaUtil):
    alpha = 3
    utilidad = [alpha**3,alpha**2,alpha,1]
    utilidadTotal = 0
    
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    
    claves = datosCarga['Clave'].unique()
    for i in range(len(claves)):
        if claves[i][0:2] == 'AD'or claves[i][0:2] == 'TA' or claves[i][0:2] == 'LI' or claves[i][0:2] == 'PI':
            continue
            
        ciclo = int(plan_2016.query('clave == "' + claves[i] + '"')['ciclos'])
        utilidadTotal += utilidad[ciclo-1]
    
    #Normalización
    utilidadNorm = (utilidadTotal)/(utilidad[0]*9)
    return utilidadNorm

## Costo de carga académica con base en las horas libres

#### NOTA: Tomar en cuenta las horas libres por día

In [ ]:
def CpHL(solucion,ofertaUtil):
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado']
    costoTotal = 0
    hlMax = 0
    
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    
    for dia in dias:
        datosCarga = datosCarga.sort_values(dia)
        if datosCarga[dia].iloc[len(datosCarga)-2] == '-':
            continue
        
        ultimaHoraFin = 0
        for i in range(len(datosCarga)):
            if datosCarga.iloc[i][dia] == '-':
                continue
            if ultimaHoraFin == 0:
                ultimaHoraFin = int(datosCarga.iloc[i][dia][6:8])
                primeraHoraDia = int(datosCarga.iloc[i][dia][0:2])
                continue
            horaInicio = int(datosCarga.iloc[i][dia][0:2])
            costoTotal += (horaInicio - ultimaHoraFin)
            ultimaHoraFin = int(datosCarga.iloc[i][dia][6:8])
        hlMax += (ultimaHoraFin - primeraHoraDia - 2)
    
    if hlMax == 0:
        return 0
    #Normalización
    costo = (costoTotal)/(hlMax)
    return costo

## Costo de carga académica con base en la disponibilidad de horario del estudiante

Creamos 4 casos diferentes donde los estudiantes trabajan. Guardamos el horario de ocupación de los estudiantes en un dataframe por cada estudiante.

In [ ]:
# El estudiante trabaja por las mañanas tiempo completo
disp_est_1 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Martes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Miercoles": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Jueves": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Viernes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Sabado": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True]
})


# El estudiante trabaja por las tardes tiempo completo
disp_est_2 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Martes": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Miercoles": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Jueves": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Viernes": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Sabado": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False]
})

# El estudiante trabaja por las mañanas medio tiempo
disp_est_3 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Martes": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Miercoles": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Jueves": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Viernes": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Sabado": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False]
})
# El estudiante trabaja por las tardes medio tiempo
disp_est_4 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Martes": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Miercoles": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Jueves": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Viernes": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Sabado": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True]
})
disp_est_5 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Martes": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Miercoles": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Jueves": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Viernes": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Sabado": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True]
})

In [ ]:
disponibilidadArr = [disp_est_1,disp_est_2,disp_est_3,disp_est_4,disp_est_5]

In [ ]:
def CpDH(solucion,ofertaUtil,disponibilidad):
    disponibilidadTotal = sum(disponibilidad['Lunes']) + sum(disponibilidad['Martes']) + sum(disponibilidad['Miercoles']) + sum(disponibilidad['Jueves']) + sum(disponibilidad['Viernes']) + sum(disponibilidad['Sabado'])
    if disponibilidadTotal == 90:
        return 0
    
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado']
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    costoTotal = 0

    for dia in dias:
        for i in range(len(datosCarga)):
            if datosCarga[dia].iloc[i] == '-':
                continue

            horaInicio = int(datosCarga.iloc[i][dia][0:2])
            horaFin = int(datosCarga.iloc[i][dia][6:8])

            for hora in range(horaInicio,horaFin):
                if not(disponibilidad.query('hora == ' + str(hora))[dia].values[0]):
                    costoTotal += 1
                    
    #Normalización
    costo = (costoTotal)/(90 - disponibilidadTotal)
    return costo

# Definición de la función de desempeño

In [ ]:
def obtenerDesempeno(solucion,cromosomaIndice):
    if not(esValido(solucion,ofertaUtil)):
        return 0
    pesos = np.array([1,1,1,1])   
    
    upcc = UpCC(solucion,ofertaUtil)
    upmr = UpMR(kardex,solucion,ofertaUtil)
    cpdh = CpDH(solucion,ofertaUtil,disponibilidad)
    cphl = CpHL(solucion,ofertaUtil)
    
    x = np.array([upcc,upmr,cpdh,cphl])
    x = x*pesos
    
    x[2] =  pesos[2]- x[2]
    x[3] = pesos[3] - x[3]
    
    return sum(x)

In [ ]:
def evaluarSolucion(solucion):
    pesos = np.array([1,1,1,1])   
    
    upcc = UpCC(solucion,ofertaUtil)
    upmr = UpMR(kardex,solucion,ofertaUtil)
    cpdh = CpDH(solucion,ofertaUtil,disponibilidad)
    cphl = CpHL(solucion,ofertaUtil)
    
    print("La carga evaluada tiene:")
    print("\t" + str(round(100*upcc,2)) + "% de utilidad por las materias de primeros ciclos llevadas.")
    print("\t" + str(round(100*upmr,2)) + "% del total de las materias reprobadas del estudiante.") 
    print("\t" + str(round(100*cpdh,2)) + "% de costo por sobreposición con las horas donde el estudiante no está disponible.")
    print("\t" + str(round(100*cphl,2)) + "% de costo por el total de horas libres por día.")

### Problemas a solucionar:

-Función de horas libres no toma en cuenta el promedio de horas libres por día (Hacer otra función?)

-¿La función objetivo de regresar un promedio o una suma de las demás funciones?

-Arreglar costo por horas libres, regresa valores negativos

# Algoritmos evolutivos

## Prueba #1

In [ ]:
#Matricula Eddie = 190311499

In [ ]:
kardex = obtenerKardex(190311499).query('periodo < 202201')
disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [ ]:
#Obtener un kardex con al menos la mitad de creditos y al menos 3 materias reprobadas
#kardex = obtenerKardex()
#while obtenerCreditos(kardex) < 200 or len(obtenerMateriasReprobadas(kardex))  <= 2:
#    kardex = obtenerKardex()
#ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [ ]:
obtenerCreditos(kardex)

In [ ]:
obtenerMateriasReprobadas(kardex)

In [ ]:
ofertaUtil

In [ ]:
fitness_function = obtenerDesempeno

num_generations = 30      #Número de generaciones
num_parents_mating = 2    #Número de padres por hijo

sol_per_pop = 500        #Tamaño de la poblacion
num_genes = 9             #Tamaño del cromosoma

gene_type = int           #Tipo de dato de los genes del cromosoma

parent_selection_type = "tournament"
keep_parents = 1          #Mantener a los padres vivos (La poblacion crece)

crossover_type = "two_points"

mutation_type = "random"
mutation_percent_genes = 15

gene_space = np.append(ofertaUtil.index.values,[-1])     #Valores que pueden tomar los genes

In [ ]:
def on_generation(ga_instance):
    if ga_instance.num_generations > 100:
        if ga_instance.generations_completed % (ga_instance.num_generations / 100) == 0:
            print(str(ga_instance.generations_completed / (ga_instance.num_generations / 100)) + "%")
    else:
        print("Generación: " + str(ga_instance.generations_completed))

In [ ]:
ga_instance = pg.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       gene_type=gene_type,
                       gene_space=gene_space,
                       on_generation=on_generation)

In [ ]:
start_time = time.time()
ga_instance.run()
print("--- Proceso terminado en: %s segundos---" % (time.time() - start_time))

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()

In [ ]:
solution_fitness

In [ ]:
ga_instance.plot_fitness()

**Nota:** Esta carga tiene un mejor resultado práctico que el resultado ideal.

In [ ]:
obtenerDatosCarga(solution,ofertaUtil)

In [ ]:
desempeno = ga_instance.last_generation_fitness
desempeno.sort()

In [ ]:
for a in desempeno:
    print(a)

In [ ]:
solution

In [ ]:
obtenerDesempeno([66,67,68,72,69,71,-1,-1,-1],0)

In [ ]:
claves = obtenerDatosCarga(solution,ofertaUtil)['Clave'].values
for clave in claves:
    print(plan_2016.query('clave == "' + clave + '"')['ciclos'].values)

In [ ]:
evaluarSolucion(solution) #Solucion del algoritmo

In [ ]:
9/243

#### Solución ideal:

In [ ]:
obtenerDatosCarga([66,67,68,72,69,71,-1,-1,-1],ofertaUtil)

In [ ]:
evaluarSolucion([66,67,68,72,69,71,-1,-1,-1]) #Solución ideal

In [ ]:
claves = obtenerDatosCarga([66,67,68,72,69,71,-1,-1,-1],ofertaUtil)['Clave'].values
for clave in claves:
    print(plan_2016.query('clave == "' + clave + '"')['ciclos'].values)

**Resultado:** Por dos horas libres se obtuvo el inverso del costo del 82%. Se está castigando mucho las horas libres, tal vez hay que hacer la función de costo por horas libres una función cuadrática.

Las materias de elección libre no deberían añadir un valor sobre la utilidad de cierre de ciclos a menos que aún no se hayan cursado las 2 materias de elib de primer y segundo ciclo.

Probar resultado ajustando el peso de cada objetivo: Disponibilidad(3), cerrar ciclos (2), materias reprobadas (2), horas libres(1)

## Prueba #2

Se quitaron las materias de elección libre de los primeros 4 semestres.

In [ ]:
kardex = obtenerKardex(190311499).query('periodo < 202201')
disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [ ]:
obtenerCreditos(kardex)

In [ ]:
obtenerMateriasReprobadas(kardex)

In [ ]:
fitness_function = obtenerDesempeno

num_generations = 30      #Número de generaciones
num_parents_mating = 2    #Número de padres por hijo

sol_per_pop = 500        #Tamaño de la poblacion
num_genes = 9             #Tamaño del cromosoma

gene_type = int           #Tipo de dato de los genes del cromosoma

parent_selection_type = "tournament"
keep_parents = 1          #Mantener a los padres vivos (La poblacion crece)

crossover_type = "two_points"

mutation_type = "random"
mutation_percent_genes = 15

gene_space = np.append(ofertaUtil.index.values,[-1])     #Valores que pueden tomar los genes

In [ ]:
ga_instance = pg.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       gene_type=gene_type,
                       gene_space=gene_space,
                       on_generation=on_generation)

In [ ]:
start_time = time.time()
ga_instance.run()
print("--- Proceso terminado en: %s segundos---" % (time.time() - start_time))

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print(solution_fitness)

In [ ]:
ga_instance.plot_fitness()

In [ ]:
obtenerDatosCarga(solution,ofertaUtil)

In [ ]:
desempeno = ga_instance.last_generation_fitness
desempeno.sort()
for a in desempeno:
    print(a)

In [ ]:
claves = obtenerDatosCarga(solution,ofertaUtil)['Clave'].values
for clave in claves:
    print(plan_2016.query('clave == "' + clave + '"')['ciclos'].values)

## Prueba #3

keep_parents = False

In [ ]:
kardex = obtenerKardex(190311499).query('periodo < 202201')
disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [ ]:
obtenerCreditos(kardex)

In [ ]:
obtenerMateriasReprobadas(kardex)

In [ ]:
fitness_function = obtenerDesempeno

num_generations = 30      #Número de generaciones
num_parents_mating = 2    #Número de padres por hijo

sol_per_pop = 500        #Tamaño de la poblacion
num_genes = 9             #Tamaño del cromosoma

gene_type = int           #Tipo de dato de los genes del cromosoma

parent_selection_type = "tournament"
keep_parents = 0          #Mantener a los padres vivos (La poblacion crece)

crossover_type = "two_points"

mutation_type = "random"
mutation_percent_genes = 15

gene_space = np.append(ofertaUtil.index.values,[-1])     #Valores que pueden tomar los genes

In [ ]:
ga_instance = pg.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       gene_type=gene_type,
                       gene_space=gene_space,
                       on_generation=on_generation)

In [ ]:
start_time = time.time()
ga_instance.run()
print("--- Proceso terminado en: %s segundos---" % (time.time() - start_time))

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print(solution_fitness)

In [ ]:
ga_instance.plot_fitness()

In [ ]:
obtenerDatosCarga(solution,ofertaUtil)

In [ ]:
desempeno = ga_instance.last_generation_fitness
desempeno.sort()
for a in desempeno:
    print(a)

In [ ]:
claves = obtenerDatosCarga(solution,ofertaUtil)['Clave'].values
for clave in claves:
    print(plan_2016.query('clave == "' + clave + '"')['ciclos'].values)

In [ ]:
evaluarSolucion(ga_instance.population[45])

In [ ]:
obtenerDesempeno(ga_instance.population[45],0)

In [ ]:
obtenerDatosCarga(ga_instance.population[25],ofertaUtil)

## Prueba #4

steady-state selection

In [ ]:
kardex = obtenerKardex(190311499).query('periodo < 202201')
disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [ ]:
obtenerCreditos(kardex)

In [ ]:
obtenerMateriasReprobadas(kardex)

In [ ]:
fitness_function = obtenerDesempeno

num_generations = 30      #Número de generaciones
num_parents_mating = 2    #Número de padres por hijo

sol_per_pop = 500        #Tamaño de la poblacion
num_genes = 9             #Tamaño del cromosoma

gene_type = int           #Tipo de dato de los genes del cromosoma

parent_selection_type = "sss"
keep_parents = 1          #Mantener a los padres vivos (La poblacion crece)

crossover_type = "two_points"

mutation_type = "random"
mutation_percent_genes = 15

gene_space = np.append(ofertaUtil.index.values,[-1])     #Valores que pueden tomar los genes

In [ ]:
ga_instance = pg.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       gene_type=gene_type,
                       gene_space=gene_space,
                       on_generation=on_generation)

In [ ]:
start_time = time.time()
ga_instance.run()
print("--- Proceso terminado en: %s segundos---" % (time.time() - start_time))

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print(solution_fitness)

In [ ]:
ga_instance.plot_fitness()

In [ ]:
obtenerDatosCarga(solution,ofertaUtil)

In [ ]:
desempeno = ga_instance.last_generation_fitness
desempeno.sort()
for a in desempeno:
    print(a)

In [ ]:
claves = obtenerDatosCarga(solution,ofertaUtil)['Clave'].values
for clave in claves:
    print(plan_2016.query('clave == "' + clave + '"')['ciclos'].values)

In [ ]:
evaluarSolucion(solution)

## Prueba #5

steady-state selection, uniform crossover

In [ ]:
kardex = obtenerKardex(190311499).query('periodo < 202201')
disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [ ]:
obtenerCreditos(kardex)

In [ ]:
obtenerMateriasReprobadas(kardex)

In [ ]:
fitness_function = obtenerDesempeno

num_generations = 30      #Número de generaciones
num_parents_mating = 2    #Número de padres por hijo

sol_per_pop = 500        #Tamaño de la poblacion
num_genes = 9             #Tamaño del cromosoma

gene_type = int           #Tipo de dato de los genes del cromosoma

parent_selection_type = "sss"
keep_parents = 1          #Mantener a los padres vivos (La poblacion crece)

crossover_type = "uniform"

mutation_type = "random"
mutation_percent_genes = 15

gene_space = np.append(ofertaUtil.index.values,[-1])     #Valores que pueden tomar los genes

In [ ]:
ga_instance = pg.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       gene_type=gene_type,
                       gene_space=gene_space,
                       on_generation=on_generation)

In [ ]:
start_time = time.time()
ga_instance.run()
print("--- Proceso terminado en: %s segundos---" % (time.time() - start_time))

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print(solution_fitness)

In [ ]:
ga_instance.plot_fitness()

In [ ]:
obtenerDatosCarga(solution,ofertaUtil)

In [ ]:
desempeno = ga_instance.last_generation_fitness
desempeno.sort()
for a in desempeno:
    print(a)

In [ ]:
claves = obtenerDatosCarga(solution,ofertaUtil)['Clave'].values
for clave in claves:
    print(plan_2016.query('clave == "' + clave + '"')['ciclos'].values)

In [ ]:
evaluarSolucion(solution)

## Prueba #6

steady-state selection, uniform crossover, adaptive mutation

In [ ]:
kardex = obtenerKardex(190311499).query('periodo < 202201')
disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [ ]:
obtenerCreditos(kardex)

In [ ]:
obtenerMateriasReprobadas(kardex)

In [ ]:
fitness_function = obtenerDesempeno

num_generations = 30      #Número de generaciones
num_parents_mating = 2    #Número de padres por hijo

sol_per_pop = 500        #Tamaño de la poblacion
num_genes = 9             #Tamaño del cromosoma

gene_type = int           #Tipo de dato de los genes del cromosoma

parent_selection_type = "sss"
keep_parents = 1          #Mantener a los padres vivos (La poblacion crece)

crossover_type = "uniform"

mutation_type = "adaptive"
mutation_probability = [0.35,0.1]

gene_space = np.append(ofertaUtil.index.values,[-1])     #Valores que pueden tomar los genes

In [ ]:
ga_instance = pg.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_probability=mutation_probability,
                       gene_type=gene_type,
                       gene_space=gene_space,
                       on_generation=on_generation)

In [ ]:
start_time = time.time()
ga_instance.run()
print("--- Proceso terminado en: %s segundos---" % (time.time() - start_time))

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print(solution_fitness)

In [ ]:
ga_instance.plot_fitness()

In [ ]:
obtenerDatosCarga(solution,ofertaUtil)

In [ ]:
desempeno = ga_instance.last_generation_fitness
desempeno.sort()
for a in desempeno:
    print(a)

In [ ]:
claves = obtenerDatosCarga(solution,ofertaUtil)['Clave'].values
for clave in claves:
    print(plan_2016.query('clave == "' + clave + '"')['ciclos'].values)

In [ ]:
evaluarSolucion(solution)

**Resultados:** El algoritmo evolutivo parece encontrar sino un óptimo global, al menos un desempeño bastante bueno para la población en general en pocas generaciones. Solamente falta comprobar si la función de desempeño es adecuada, para hay que evaluar los "mejores resultados".

## Prueba 7

In [ ]:
kardex = obtenerKardex(190311499).query('periodo < 202201')
disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [ ]:
obtenerCreditos(kardex)

In [ ]:
obtenerMateriasReprobadas(kardex)

In [ ]:
fitness_function = obtenerDesempeno

num_generations = 30      #Número de generaciones
num_parents_mating = 2    #Número de padres por hijo

sol_per_pop = 500        #Tamaño de la poblacion
num_genes = 9             #Tamaño del cromosoma

gene_type = int           #Tipo de dato de los genes del cromosoma

parent_selection_type = "sss"
keep_parents = 1          #Mantener a los padres vivos (La poblacion crece)

crossover_type = "single_point"

mutation_type = "adaptive"
mutation_probability = [0.35,0.1]

gene_space = np.append(ofertaUtil.index.values,[-1])     #Valores que pueden tomar los genes

In [ ]:
ga_instance = pg.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_probability=mutation_probability,
                       gene_type=gene_type,
                       gene_space=gene_space,
                       on_generation=on_generation)

In [ ]:
start_time = time.time()
ga_instance.run()
print("--- Proceso terminado en: %s segundos---" % (time.time() - start_time))

In [ ]:
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print(solution_fitness)

In [ ]:
ga_instance.plot_fitness()

In [ ]:
obtenerDatosCarga(solution,ofertaUtil)

In [ ]:
desempeno = ga_instance.last_generation_fitness
desempeno.sort()
for a in desempeno:
    print(a)

In [ ]:
claves = obtenerDatosCarga(solution,ofertaUtil)['Clave'].values
for clave in claves:
    print(plan_2016.query('clave == "' + clave + '"')['ciclos'].values)

In [ ]:
evaluarSolucion(solution)

# Selección de mejor algoritmo

Para encontrar el mejor algoritmo vamos a probar 100 kardex diferentes y aplicar cada algoritmo con un total de 20 generaciones. Posteriormente evaluaremos los mejores desempeños de cada algoritmo aplicado a cada kardex para identificar el que obtuvo mejores resultados en promedio.

In [ ]:
matriculas_set = set()
while len(matriculas_set) < 1:
    matriculas_set.add(np.random.choice(matriculas))

In [ ]:
kardexArr = []
for matricula in matriculas_set:
    kardexArr.append(obtenerKardex(matricula))

In [ ]:
type(kardexArr[2])

In [ ]:
num_generations = 15



'''
instancia1 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='uniform',
                       mutation_type='random',
                       mutation_percent_genes=15,
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

instancia2 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='single_point',
                       mutation_type='random',
                       mutation_percent_genes=15,
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

instancia3 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='two_points',
                       mutation_type='random',
                       mutation_percent_genes=15,
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

instancia4 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='uniform',
                       mutation_type = "adaptive",
                       mutation_probability = [0.35,0.1],
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

instancia5 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='single_point',
                       mutation_type = "adaptive",
                       mutation_probability = [0.35,0.1],
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

instancia6 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='two_points',
                       mutation_type = "adaptive",
                       mutation_probability = [0.35,0.1],
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))
'''

soluciones1 = []
desempenos1 = []
generaciones1 = []
tiempos1 = []

soluciones2 = []
desempenos2 = []
tiempos2 = []
generaciones2 = []

soluciones3 = []
desempenos3 = []
generaciones3 = []
tiempos3 = []

soluciones4 = []
desempenos4 = []
tiempos4 = []
generaciones4 = []

soluciones5 = []
desempenos5 = []
generaciones5 = []
tiempos5 = []

soluciones6 = []
desempenos6 = []
generaciones6 = []
tiempos6 = []

for kardex in kardexArr:
    print('Iniciando proceso de matricula: ' + str(kardex.iloc[0]["n_matricula"]))
    disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
    ofertaUtil = obtenerOfertaUtil(kardex,oferta)

    instancia1 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='uniform',
                       mutation_type='random',
                       mutation_percent_genes=15,
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia1.run()
    tiempoejecucion = time.time() - start_time
    tiempos1.append(tiempoejecucion)
    generacionDeMejorSolucion = instancia1.best_solution_generation
    solution, solution_fitness, solution_idx = instancia1.best_solution()
    soluciones1.append(solution)
    desempenos1.append(solution_fitness)
    generaciones1.append(generacionDeMejorSolucion)
    print("--- Instancia 1 terminada en: %s segundos---" % (tiempoejecucion))
    print("Desempeño: " + str(solution_fitness))
    print("Solucion: " + str(solution))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion))

    
    disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
    ofertaUtil = obtenerOfertaUtil(kardex,oferta)

    instancia2 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='single_point',
                       mutation_type='random',
                       mutation_percent_genes=15,
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))


    start_time = time.time()
    instancia2.run()
    tiempoejecucion = time.time() - start_time
    tiempos2.append(tiempoejecucion)

    generacionDeMejorSolucion = instancia2.best_solution_generation
    solution, solution_fitness, solution_idx = instancia2.best_solution()
    soluciones2.append(solution)
    desempenos2.append(solution_fitness)
    generaciones2.append(generacionDeMejorSolucion)
    print("--- Instancia 2 terminada en: %s segundos---" % (tiempoejecucion))
    print("Desempeño: " + str(solution_fitness))
    print("Solucion: " + str(solution))
    print("Mejor generacion: " + str(generacionDeMejorSolucion))
    
    disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
    ofertaUtil = obtenerOfertaUtil(kardex,oferta)
    
    instancia3 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='two_points',
                       mutation_type='random',
                       mutation_percent_genes=15,
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia3.run()
    tiempoejecucion = time.time() - start_time
    tiempos3.append(tiempoejecucion)

    generacionDeMejorSolucion = instancia3.best_solution_generation
    solution, solution_fitness, solution_idx = instancia3.best_solution()
    soluciones3.append(solution)
    desempenos3.append(solution_fitness)
    generaciones3.append(generacionDeMejorSolucion)
    print("--- Instancia 3 terminada en: %s segundos---" % (tiempoejecucion))
    print("Desempeño: " + str(solution_fitness))
    print("Solucion: " + str(solution))
    print("Mejor generacion: " + str(generacionDeMejorSolucion))
    

    disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
    ofertaUtil = obtenerOfertaUtil(kardex,oferta)

    instancia4 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='uniform',
                       mutation_type = "adaptive",
                       mutation_probability = [0.35,0.1],
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia4.run()
    tiempoejecucion = time.time() - start_time
    tiempos4.append(tiempoejecucion)

    generacionDeMejorSolucion = instancia4.best_solution_generation
    solution, solution_fitness, solution_idx = instancia4.best_solution()
    soluciones4.append(solution)
    desempenos4.append(solution_fitness)
    generaciones4.append(generacionDeMejorSolucion)
    print("--- Instancia 4 terminada en: %s segundos---" % (tiempoejecucion))
    print("Desempeño: " + str(solution_fitness))
    print("Solucion: " + str(solution))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion))
    

    disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
    ofertaUtil = obtenerOfertaUtil(kardex,oferta)

    instancia5 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='single_point',
                       mutation_type = "adaptive",
                       mutation_probability = [0.35,0.1],
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia5.run()
    tiempoejecucion = time.time() - start_time
    tiempos5.append(tiempoejecucion)

    generacionDeMejorSolucion = instancia5.best_solution_generation
    solution, solution_fitness, solution_idx = instancia5.best_solution()
    soluciones5.append(solution)
    desempenos5.append(solution_fitness)
    generaciones5.append(generacionDeMejorSolucion)
    print("--- Instancia 5 terminada en: %s segundos---" % (tiempoejecucion))
    print("Desempeño: " + str(solution_fitness))
    print("Solucion: " + str(solution))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion))
    

    disponibilidad = disponibilidadArr[4] #Disponibilidad completa de horario
    ofertaUtil = obtenerOfertaUtil(kardex,oferta)

    instancia6 = pg.GA(num_generations=num_generations,
                       num_parents_mating=2,
                       fitness_func=obtenerDesempeno,
                       sol_per_pop=500,
                       num_genes=9,
                       parent_selection_type="sss",
                       keep_parents=1,
                       crossover_type='two_points',
                       mutation_type = "adaptive",
                       mutation_probability = [0.35,0.1],
                       gene_type=int,
                       gene_space=np.append(ofertaUtil.index.values,[-1]))

    start_time = time.time()
    instancia6.run()
    tiempoejecucion = time.time() - start_time
    tiempos6.append(tiempoejecucion)

    generacionDeMejorSolucion = instancia6.best_solution_generation
    solution, solution_fitness, solution_idx = instancia6.best_solution()
    soluciones6.append(solution)
    desempenos6.append(solution_fitness)
    generaciones6.append(generacionDeMejorSolucion)
    print("--- Instancia 6 terminada en: %s segundos---" % (tiempoejecucion))
    print("Desempeño: " + str(solution_fitness))
    print("Solucion: " + str(solution))
    print("Mejor generacion: "+ str(generacionDeMejorSolucion))

print("\n\nPROCESO DE MATRICULA TERMINADO EN " + str(''))

In [ ]:
solution, solution_fitness, solution_idx = instancia1.best_solution()

In [ ]:
solution

In [ ]:
solution_fitness

In [ ]:
instancia1.plot_fitness()

In [ ]:
instancia2.plot_fitness()

In [ ]:
instancia3.plot_fitness()

In [ ]:
instancia4.plot_fitness()

In [ ]:
instancia5.plot_fitness()

In [ ]:
instancia6.plot_fitness()